# Playing with ansible API

- https://docs.ansible.com/ansible/dev_guide/developing_api.html

- https://www.ansible.com/blog/how-to-extend-ansible-through-plugins — excellent overview of ansible extension points

- <img alt="Extending Ansible cover" src="Extending_Ansible_cover.jpg" style="width: 20%; float: right">
  [*Extending Ansible* book][3] by Rishabh Das (2016, I think describes pre-2.0?)  
  Free sample including API chapter at https://www.ansible.com/extending-ansible

- [`lib/ansible/adhoc.py`][1] and [`lib/ansible/playbook.py`][2] are simple usage examples.

[1]: https://github.com/ansible/ansible/blob/devel/lib/ansible/cli/adhoc.py
[2]: https://github.com/ansible/ansible/blob/devel/lib/ansible/cli/playbook.py
[3]: https://www.packtpub.com/networking-and-servers/extending-ansible

In [1]:
from ansible import constants as C
from ansible.cli import CLI
from ansible.errors import AnsibleError, AnsibleOptionsError, AnsibleParserError
from ansible.executor.task_queue_manager import TaskQueueManager
from ansible.inventory import Inventory
#from ansible.module_utils._text import to_text
from ansible.parsing.dataloader import DataLoader
#from ansible.parsing.splitter import parse_kv
from ansible.playbook.play import Play
#from ansible.plugins import get_all_plugin_loaders
#from ansible.utils.vars import load_extra_vars
#from ansible.utils.vars import load_options_vars
from ansible.vars import VariableManager


In [2]:
variable_manager = VariableManager()

In [3]:
loader = DataLoader()

In [4]:
inventory = Inventory(loader=loader, variable_manager=variable_manager)

In [5]:
passwords = {}

An `options` object is needed, many places in code require specific attributes to exist.
Could build one but easier to use the CLI arguments parser to provide them.

In [6]:
#import argparse
#options = argparse.Namespace(module_path=None, forks=C.DEFAULT_FORKS, become=C.DEFAULT_BECOME)
parser = CLI.base_parser(module_opts=True, fork_opts=True, runas_opts=True, check_opts=True)
options, extra_args = parser.parse_args([])
options

<Values at 0x7f9197da47b8: {'su': False, 'ask_sudo_pass': False, 'become_method': 'sudo', 'diff': False, 'forks': 5, 'become_ask_pass': False, 'become_user': None, 'sudo_user': None, 'become': False, 'check': False, 'ask_su_pass': False, 'module_path': None, 'sudo': False, 'syntax': None, 'verbosity': 0, 'su_user': None}>

In [7]:
def task_queue_manager():
    return TaskQueueManager(
        inventory=inventory,
        variable_manager=variable_manager,
        loader=loader,
        options=options,
        passwords=passwords,
        #stdout_callback=cb,
        #run_additional_callbacks=C.DEFAULT_LOAD_CALLBACK_PLUGINS,
        #run_tree=run_tree,
    )
tqm = task_queue_manager()

## Getting a Play data structure

In [8]:
import yaml

In [9]:
play1 = Play.load(yaml.load('''
hosts: localhost
tasks:
  - command: zenity --question --text="WORKS! Proceed?"
'''))
play1.tasks

[BLOCK(uuid=68f728fa-c3ab-235d-5a18-000000000002)(id=140263294266392)(parent=None)]

In [10]:
tqm.run(play1)


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [command] *****************************************************************
changed: [localhost]


0

## TaskQueueManager is stateful
Look what happens after a play fails:

In [19]:
failing_play = Play.load(yaml.load('''
hosts: localhost
tasks:
  - command: 'false'
'''))
tqm.run(failing_play)


PLAY [localhost] ***************************************************************


2

In [20]:
tqm.run(play1)


PLAY [localhost] ***************************************************************


2

 => tqm will not run anything more, it will just return error exit code :-(

In [21]:
TaskQueueManager.RUN_FAILED_HOSTS

2

### Make it easier to run

In [22]:
def run(code):
    task_queue_manager().run(Play.load(yaml.load(code)))

In [23]:
run("""
tasks:
  - command: echo foo
""")

AnsibleParserError: the field 'hosts' is required but was not set

Oops.  That was not convenient enough.  Also, what a huge stacktrace :-
(

In [24]:
import sys
def run(code):
    try:
        data = yaml.load(code)
        play = Play.load(data)
        task_queue_manager().run(play)
    except (yaml.YAMLError, AnsibleParserError) as e:
        print(e, file=sys.stderr)

In [25]:
run("""
tasks
- command: echo foo
""")

mapping values are not allowed here
  in "<unicode string>", line 3, column 10:
    - command: echo foo
             ^


In [26]:
run("""
tasks:
- command: echo foo
""")

the field 'hosts' is required but was not set


In [27]:
def to_play(code):
    """Support one task, list of tasks, or whole play without hosts."""
    data = yaml.load(code)
    if isinstance(data, dict) and 'tasks' not in data:
        data = [data]
    if isinstance(data, list):
        data = dict(tasks=data)
    if 'hosts' not in data:
        data['hosts'] = 'localhost'
    return Play.load(data)

def run(code):
    try:
        task_queue_manager().run(to_play(code))
    except (yaml.YAMLError, AnsibleParserError) as e:
        print(e, file=sys.stderr)


In [28]:
run("""command: echo foo""")


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [command] *****************************************************************
changed: [localhost]


In [29]:
run("""set_fact: var=1""")


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [set_fact] ****************************************************************
ok: [localhost]


In [30]:
run("""debug: msg={{var}}""")


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "changed": false,
    "msg": "1"
}


## How is a play processed before execution?
### Let's consider a more complex play.

In [31]:
play = to_play('''
vars:
  play_local_variable: 'abc'
  
tasks:

- debug: 'msg=The var equals {{play_local_variable}}'

- command: pwd
  register: pwd_out  # Sets a variable

- set_fact: global1=value1 global2=value2  # Some more vars

- with_items: [play_local_variable, pwd_out, global1, global2]  # A loop!
  debug: var={{item}}
''')
task_queue_manager().run(play)


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => {
    "changed": false,
    "msg": "The"
}

TASK [command] *****************************************************************
changed: [localhost]

TASK [set_fact] ****************************************************************
ok: [localhost]

TASK [debug] *******************************************************************
ok: [localhost] => (item=play_local_variable) => {
    "item": "play_local_variable",
    "play_local_variable": "abc"
}
ok: [localhost] => (item=pwd_out) => {
    "item": "pwd_out",
    "pwd_out": {
        "changed": true,
        "cmd": [
            "pwd"
        ],
        "delta": "0:00:00.003555",
        "end": "2017-06-11 10:37:23.649413",
        "rc": 0,
        "start": "2017-06

0

*__Tip__: Ansible objects have `.serialize()`, handy for exploring.*

In [32]:
play.serialize()

{'accelerate': False,
 'accelerate_ipv6': False,
 'accelerate_port': 5099,
 'always_run': None,
 'any_errors_fatal': False,
 'become': None,
 'become_flags': None,
 'become_method': None,
 'become_user': None,
 'check_mode': None,
 'connection': None,
 'environment': None,
 'fact_path': None,
 'finalized': False,
 'force_handlers': None,
 'gather_facts': None,
 'gather_subset': None,
 'gather_timeout': None,
 'handlers': [],
 'hosts': 'localhost',
 'ignore_errors': None,
 'included_path': None,
 'max_fail_percentage': None,
 'name': 'localhost',
 'no_log': None,
 'port': None,
 'post_tasks': [],
 'pre_tasks': [],
 'remote_user': None,
 'roles': [],
 'run_once': None,
 'serial': [],
 'squashed': False,
 'strategy': 'linear',
 'tags': [],
 'tasks': [BLOCK(uuid=68f728fa-c3ab-235d-5a18-0000000000e1)(id=140263238199616)(parent=None)],
 'uuid': '68f728fa-c3ab-235d-5a18-0000000000e0',
 'vars': {'play_local_variable': 'abc'},
 'vars_files': [],
 'vars_prompt': [],
 'vault_password': None}

In [33]:
play.get_vars()

{'play_local_variable': 'abc'}

In [34]:
variable_manager.get_vars(loader=loader, play=play)

{'ansible_playbook_python': '/usr/bin/python3',
 'ansible_version': {'full': '2.3.0.0',
  'major': 2,
  'minor': 3,
  'revision': 0,
  'string': '2.3.0.0'},
 'hostvars': {},
 'omit': '__omit_place_holder__fbafc4fa7c4040051c81cca43b7872f07e0bc7f6',
 'play_local_variable': 'abc',
 'playbook_dir': '.',
 'role_names': [],
 'vars': {'ansible_playbook_python': '/usr/bin/python3',
  'ansible_version': {'full': '2.3.0.0',
   'major': 2,
   'minor': 3,
   'revision': 0,
   'string': '2.3.0.0'},
  'hostvars': {},
  'omit': '__omit_place_holder__fbafc4fa7c4040051c81cca43b7872f07e0bc7f6',
  'play_local_variable': 'abc',
  'playbook_dir': '.',
  'role_names': []}}

In [35]:
play.compile()

[BLOCK(uuid=68f728fa-c3ab-235d-5a18-00000000010a)(id=140263237750008)(parent=None),
 BLOCK(uuid=68f728fa-c3ab-235d-5a18-0000000000e1)(id=140263238199616)(parent=None),
 BLOCK(uuid=68f728fa-c3ab-235d-5a18-00000000010a)(id=140263237750008)(parent=None),
 BLOCK(uuid=68f728fa-c3ab-235d-5a18-00000000010a)(id=140263237750008)(parent=None)]

^^ These are the pre_tasks, roles, tasks, and post_tasks

In [38]:
[list(block.block) for block in play.compile()]


[[TASK: meta (flush_handlers)],
 [TASK: debug, TASK: command, TASK: set_fact, TASK: debug],
 [TASK: meta (flush_handlers)],
 [TASK: meta (flush_handlers)]]

In [41]:
play.compile()[1].serialize()

TypeError: 'Block' object is not iterable

## Variables

In [ ]:
from ansible.vars import preprocess_vars
preprocess_vars(yaml.load('''
hosts: localhost
tasks:
  - command: 'false'
'''))

# Peek under the hood of IPyKernel running *this* notebook?

In [ ]:
import sys
sorted(sys.modules)

In [ ]:
import ipykernel.ipkernel

In [ ]:
import gc
[obj for obj in gc.get_referrers(ipykernel.ipkernel.IPythonKernel) 
 if isintance(obj,ipykernel.ipkernel.IPythonKernel)]